In [12]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)
    
N_EPOCHS = 50
N_TRIALS = 10
BATCH_SIZE = 512

CONFIG = config_tests["proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
ds_1 = tfds.load("stl10", split="train", as_supervised=True, shuffle_files=False)
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)

x = []
y = []
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

2024-04-27 20:32:32.784355: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-27 20:32:33.578237: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(9100, 1950, 1950)

In [24]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [17]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [1]:
model_scores = []
verbose = 0
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
    
    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.25
    
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(2 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_proba = utils.calculate_proba(
            model, x_train_sorted, y_train_sorted, counts
        )
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=verbose,
        )
        
        for j in range(len(model.layers[:-1])):
            model.layers[j].layers[-1].rate -= 0.003
    
    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, callbacks=[early_stopping], verbose=verbose)
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

NameError: name 'tf' is not defined

In [41]:
model_scores_random = []
verbose = 0
early_stopping_random = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.3

    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, verbose=verbose, callbacks=[early_stopping_random])

    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores_random.append(accuracy)
    print("Mean:", np.mean(model_scores_random), " Median: ", np.median(model_scores_random))

 10%|█         | 1/10 [00:42<06:18, 42.09s/it]

Mean: 0.5189743638038635  Median:  0.5189743638038635


 20%|██        | 2/10 [01:13<04:48, 36.10s/it]

Mean: 0.4933333396911621  Median:  0.4933333396911621


 30%|███       | 3/10 [01:49<04:11, 35.99s/it]

...

Mean: 0.4671794958412647  Median:  0.46948719024658203


 90%|█████████ | 9/10 [05:04<00:33, 33.03s/it]

Mean: 0.4665527145067851  Median:  0.4676923155784607


100%|██████████| 10/10 [05:37<00:00, 33.76s/it]

Mean: 0.4645641088485718  Median:  0.4646153897047043


In [33]:
model_scores_random

[0.519487202167511,
 0.47846153378486633,
 0.508717954158783,
 0.4558974504470825,
 0.45384615659713745]

In [35]:
np.mean(model_scores), np.mean(model_scores_random)

(0.5317948818206787, 0.48328205943107605)

In [34]:
np.mean(sorted(model_scores)[5:]), np.mean(sorted(model_scores_random)[5:])

/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(nan, nan)

In [36]:
import scipy.stats
scipy.stats.ttest_ind(model_scores, model_scores_random)

TtestResult(statistic=3.476015034822166, pvalue=0.008368460131238117, df=8.0)